BASE CALLING SCRATCH CODE
=====

In [1]:
import itertools

bases = ['A','T','G','C']
genotypes = list(itertools.combinations_with_replacement(bases,bases))
#genotypes = [('A','A'),('A','T')]

In [2]:
from math import log10, factorial
from functools import reduce
import operator
from collections import defaultdict

# HELPER FUNCTIONS

# product of list
def prod(iterable): # credit to: http://stackoverflow.com/questions/7948291/is-there-a-built-in-product-in-python
    return reduce(operator.mul, iterable, 1)

# sum of two probabilities that are in log form
def addlogs(a,b):
    if a > b:
        return a + log10(1 + pow(10, b - a))
    else:
        return b + log10(1 + pow(10, a - b))

# difference of two probabilities that are in log form
def subtractlogs(a,b):
    if a > b:
        return a + log10(1 - pow(10, b - a))
    else:
        return b + log10(1 - pow(10, a - b))

p_null = 0.5 # probability that strand is not sampled in chamber. hardcoded until we can estimate it

# helper data structures that pre-compute probabilities of strand configurations

ch_priors = [log10((1-p_null)/24)]*24 + [log10(p_null)]
poss_ch = list(range(0,25)) # the last element is "no chamber"
het_config_probs = dict()

for c1,c2,c3,c4 in itertools.product(poss_ch,poss_ch,poss_ch,poss_ch):
    # because of symmetry we ignore cases where we can flip the strands for the same result
    # e.g. we ignore cases where strand 1 is in a later chamber than strand 2 and say that
    # the cases where it is in an earlier chamber are twice as likely. Same for strands 3,4
    if c1 > c2 and c3 > c4:
        perm = log10(4)
    elif (c1 > c2 and c3 == c4) or (c1 == c2 and c3 > c4):
        perm = log10(2)
    elif c1 == c2 and c3 == c4:
        perm = log10(1)
    else:
        continue

    het_config_probs[(c1,c2,c3,c4)] = perm+ch_priors[c1]+ch_priors[c2]+ch_priors[c3]+ch_priors[c4] # probability of configuration
    
possible_chambers = list(range(0,25))

hom_config_probs = dict()

for c1,c2,c3,c4 in itertools.product(poss_ch,poss_ch,poss_ch,poss_ch):
    
    lst  = sorted([c1,c2,c3,c4])
    if tuple(lst) in hom_config_probs:
        continue
    uniq = list(set(lst))
    counts  = [lst.count(u) for u in uniq]
    perm = log10(factorial(len(lst))/prod([factorial(c) for c in counts]))
    
    hom_config_probs[tuple(lst)] = perm+ch_priors[c1]+ch_priors[c2]+ch_priors[c3]+ch_priors[c4] # probability of configuration    
    
# INPUT
# G: a tuple that specifies genotype e.g. ('A','T')
# nonzero_chambers: a list containing the indices of chambers 0..23 (or 24, unsampled) where strands may be (have read coverage)
# OUTPUT
# a list of configurations represented by tuples.
# the tuples contain (sl,prob) where sl is a 4-tuple of strand chambers (0..23 or 24) and prob is log probability of config occuring
def singlecell_config(G,nonzero_chambers):
    g1,g2 = G
    nz = nonzero_chambers + [24]
    nzs = set(nonzero_chambers)
    
    configs = []
    for c1,c2,c3,c4 in itertools.product(nz,nz,nz,nz):
        if set.intersection({c1,c2,c3,c4},nzs) != nzs:
            continue

        # c1..c4 represent the chamber placements of strands 1..4
        sl = (c1,c2,c3,c4)
        if (g1 == g2 and sl not in hom_config_probs) or (g1 != g2 and sl not in het_config_probs):
            continue
        
        if g1 == g2:
            prob = hom_config_probs[(c1,c2,c3,c4)]
        else:
            prob = het_config_probs[(c1,c2,c3,c4)]
            
        configs.append((sl,prob))

    total = configs[0][1]
    for i in range(1,len(configs)):
        total = addlogs(total,configs[i][1])
    
    for i in range(len(configs)):
        configs[i] = (configs[i][0], configs[i][1] - total)
        
    return configs

# INPUT
# G: a tuple that specifies genotype e.g. ('A','T')
# nonzero_chambers: a list containing one list per cell.
# the inner list should contain the indices of chambers 0..23 (or 24, unsampled) where strands are found
# OUTPUT
# a list of configurations represented by tuples.
# the tuple contains one inner tuple per cell.
# these inner tuples contain (sl,prob) where sl is a 4-tuple of strand chambers (0..23 or 24) and prob is log probability of config occuring
def multicell_config(G,nonzero_chambers):
    g1,g2 = G
    
    config_sets = [singlecell_config(G,nz) for nz in nonzero_chambers]
    return itertools.product(*config_sets)
    


In [15]:
-

In [16]:
mixed_alleles = list(set([tuple(sorted(list(x))) for x in itertools.product(alleles,alleles)]))

In [17]:
mixed_alleles

[('C', 'T'),
 ('G', 'G'),
 ('C', 'C'),
 ('A', 'G'),
 ('A', 'C'),
 ('T', 'T'),
 ('A', 'A'),
 ('C', 'G'),
 ('G', 'T'),
 ('A', 'T')]

In [18]:
nz = [0,1,2,24]
it = itertools.product(nz,nz,nz,nz)

In [19]:
next(it)

(0, 0, 0, 0)

In [23]:
total = 0
for k,v in mixed_allele_priors[4].items():
    print("{}\t{}".format(k,10**v))
    total += 10**v
print(total)

('G',)	0.24999999999999986
('C', 'T')	0.0
('A', 'T')	0.0
('T',)	0.2499999999999998
('A', 'G')	0.0
('G', 'T')	0.0
('A', 'C')	0.0
('C',)	0.24999999999999967
('A',)	0.2499999999999998
('C', 'G')	0.0
0.9999999999999991


defaultdict(float,
            {0.001949317738791423: 0.010000000000000002,
             0.0038910505836575876: 0.020000000000000004,
             0.007751937984496124: 0.030000000000000006,
             0.015384615384615385: 0.04000000000000001,
             0.030303030303030304: 0.05000000000000001,
             0.058823529411764705: 0.06000000000000001,
             0.1111111111111111: 0.07,
             0.2: 0.08000000000000002,
             0.3333333333333333: 0.09000000000000002,
             0.5: 0.10000000000000003,
             0.6666666666666666: 0.09000000000000002,
             0.8: 0.08000000000000002,
             0.8888888888888888: 0.07,
             0.9411764705882353: 0.06000000000000001,
             0.9696969696969697: 0.05000000000000001,
             0.9846153846153847: 0.04000000000000001,
             0.9922480620155039: 0.030000000000000006,
             0.9961089494163424: 0.020000000000000004,
             0.9980506822612085: 0.010000000000000002})

In [31]:
P_plst = list(P_p.items())

In [33]:
P_plst.sort()

In [34]:
P_plst

[(0.001949317738791423, 0.010000000000000002),
 (0.0038910505836575876, 0.020000000000000004),
 (0.007751937984496124, 0.030000000000000006),
 (0.015384615384615385, 0.04000000000000001),
 (0.030303030303030304, 0.05000000000000001),
 (0.058823529411764705, 0.06000000000000001),
 (0.1111111111111111, 0.07),
 (0.2, 0.08000000000000002),
 (0.3333333333333333, 0.09000000000000002),
 (0.5, 0.10000000000000003),
 (0.6666666666666666, 0.09000000000000002),
 (0.8, 0.08000000000000002),
 (0.8888888888888888, 0.07),
 (0.9411764705882353, 0.06000000000000001),
 (0.9696969696969697, 0.05000000000000001),
 (0.9846153846153847, 0.04000000000000001),
 (0.9922480620155039, 0.030000000000000006),
 (0.9961089494163424, 0.020000000000000004),
 (0.9980506822612085, 0.010000000000000002)]

In [12]:
import pickle
from collections import defaultdict

#P_parent1_lst = [(0.001,log10(0.495)),(0.5,log10(0.01)),(0.999,log10(0.495))]
cov_frac_dist_raw = pickle.load(open( "parameters/cov_frac_dist.p", "rb"))
cov_frac_dist = defaultdict(list)

lim = 30
for i in range(1,lim+1):
    cov_frac_dist[i] = cov_frac_dist_raw[i]

def chunks(l, n): # credit to http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# given a list with length N*binsize, sum consectutive binsize-sized chunks into a length N list
def condense(l, binsize):
    return [sum(x) for x in chunks(l,binsize)]
    
for i in range(lim,lim):
    binsize = int(i / lim)
    lst = condense(cov_frac_dist_raw[i], binsize)
    for j in range(i,i+lim):
        cov_frac_dist[j] = lst

for i in range(lim,2000):
    print("{} {} {}".format(i,len(cov_frac_dist[i]), lim))
    assert(len(cov_frac_dist[i]) == lim)

30 30 30


AssertionError: 

In [7]:
lst

[0.0,
 0.0,
 0.10433744096411704,
 0.056285398910663552,
 0.048079184988147589,
 0.035367243906411161,
 0.03527676745743083,
 0.025758645024700072,
 0.029106273636972296,
 0.026301503718582052,
 0.025785787959394169,
 0.025034833432857426,
 0.026310551363480088,
 0.024817689955304634,
 0.02479959466550857,
 0.021334346669561913,
 0.025025785787959394,
 0.024808642310406602,
 0.026708647738993539,
 0.024464831804281346,
 0.026337694298174185,
 0.026455313681848613,
 0.028138175632882761,
 0.025686263865515806,
 0.036398675424786926,
 0.036244865461520365,
 0.048902520673868591,
 0.057588259775980311,
 0.10464506089065016,
 0.0]

In [9]:
list(range(2,20,4))

[2, 6, 10, 14, 18]

In [52]:
import itertools
import math
from math import log10
import pickle
from collections import defaultdict
from scipy.stats import chisquare

chroms = ['chr{}'.format(i) for i in range(1,23)] + ['chrX','chrY']
cells = ['PGP1_21','PGP1_22','PGP1_A1']
n_chambers = 24
chambers = list(range(1,n_chambers+1))
bases = ['A','T','G','C']
genotypes = list(itertools.combinations_with_replacement(bases,2))
parameters_dir = 'parameters'
# sum of two probabilities that are in log form
def addlogs(a,b):
    if a > b:
        return a + log10(1 + pow(10, b - a))
    else:
        return b + log10(1 + pow(10, a - b))

###############################################################################
# TEMPORARILY HARDCODED (need to be estimated form data in final version)
###############################################################################

#omega = pickle.load(open("parameters/MDA_error_rate.p", "rb"))
grams_DNA_before_MDA  = 0.25e-12  # 0.25 pg
grams_DNA_after_MDA = 6e-9        # 6 ng
GAIN = grams_DNA_after_MDA / grams_DNA_before_MDA
omega = log10(3.2e-6 / 2 * math.log2(GAIN)) # probability of MDA error.
# formula from this paper: http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0105585

#p_null = 0.5          # probability that strand is not sampled in chamber. hardcoded until we can estimate it

het_snp_rate = 0.0005
hom_snp_rate = 0.001

n_cells = 3
alleles = ['A','T','G','C']
#allele_percents = [27.25,18.90,18.91,27.29]  # percentages of bases from hg19, http://seqanswers.com/forums/archive/index.php/t-12359.html
#allele_priors   = [log10(a/sum(allele_percents)) for a in allele_percents]
mixed_alleles = list(set([tuple(sorted(list(set(x)))) for x in itertools.product(alleles,alleles)]))
#azip = list(zip(alleles, allele_priors))
transition = {'A':'G','G':'A','T':'C','C':'T'}

total_sampled_cell_positions = pickle.load(open("{}/total_sampled_cell_positions.p".format(parameters_dir), "rb"))#genome_size*sample_rate
# estimate proportions of strands in chambers
chamber_position_counts = pickle.load(open("{}/chamber_position_counts.p".format(parameters_dir), "rb" ))
total = sum(list(chamber_position_counts.values()))
chamber_proportions = [chamber_position_counts[chamber]/total for chamber in range(0,24)]
chamber_proportions = [x if x != 0 else 1e-10 for x in chamber_proportions]
total = sum(chamber_proportions)
chamber_proportions = [x/total for x in chamber_proportions]

                       
# estimate p_null, the probability of a strand not being sampled.
# we'll iterate over many possible values and choose the one that minimizes p-value for chi-square goodness of fit
#genome_size = 2897310462
strand_coverage_counts = pickle.load(open("{}/strand_coverage_counts.p".format(parameters_dir), "rb" ))


p_null = 0.75
ch_priors = [log10((1-p_null)/24)]*24 + [log10(p_null)]

max_pval = 0
sample_rate = 0.001
total_coverage = sum(list(strand_coverage_counts.values()))
observed_proportions = [strand_coverage_counts[i]/total_coverage for i in range(0,5)]

for i in range(1,100):
    
    putative_p_null = i*0.01
    putative_ch_priors = [log10((1-putative_p_null) * x) for x in chamber_proportions] + [log10(putative_p_null)]   # prior probability of sampling a given chamber

    # given these proportions, compute the expected proportions of positions
    # with a given number of strands present.
    
    poss_ch = list(range(0,25)) # the last element is "no chamber"
    coverage_dict = dict()
    for j in range(0,5):
        coverage_dict[j] = -1e100
    seen_cfgs = set()
    plist = []
    for c1,c2,c3,c4 in itertools.product(poss_ch,poss_ch,poss_ch,poss_ch):
        # because of symmetry we ignore cases where we can flip the strands for the same result
        # e.g. we ignore cases where strand 1 is in a later chamber than strand 2 and say that
        # the cases where it is in an earlier chamber are twice as likely. Same for strands 3,4
        if c1 > c2:
            temp = c1
            c1 = c2
            c2 = temp
    
        if c3 > c4:
            temp = c3
            c3 = c4
            c4 = temp  
    
        if (c1,c2,c3,c4) in seen_cfgs:
            continue
        
        if c1 < c2 and c3 < c4:
            perm = log10(4)
        elif (c1 < c2 and c3 == c4) or (c1 == c2 and c3 < c4):
            perm = log10(2)
        elif c1 == c2 and c3 == c4:
            perm = log10(1)
        else:
            continue
        seen_cfgs.add((c1,c2,c3,c4))
        strands_present = 4
        for c in [c1,c2,c3,c4]:
            if c == 24:
                strands_present -= 1
                
        cfg_prob = perm+putative_ch_priors[c1]+putative_ch_priors[c2]+putative_ch_priors[c3]+putative_ch_priors[c4]
        coverage_dict[strands_present] = addlogs(coverage_dict[strands_present],cfg_prob) # probability of configuration
        
    expected_proportions = [10**coverage_dict[j] for j in range(0,5)]
    e = expected_proportions
    chisq, chisq_pval = chisquare(observed_proportions, expected_proportions)
    print("{} {} {} {} {} | {}".format(e[0],e[1],e[2],e[3],e[4],chisq_pval))
                          
    if chisq_pval > max_pval:
        max_pval = chisq_pval
        p_null = putative_p_null
        ch_priors = putative_ch_priors

1e-08 3.9600000000000315e-06 0.0005880599999999991 0.03881195999999959 0.9605960099999944 | 0.0
1.6000000000000008e-07 3.136000000000023e-05 0.0023049599999999666 0.07529535999999742 0.9223681599999938 | 0.0
8.099999999999996e-07 0.00010476000000000055 0.005080859999999927 0.10952075999999707 0.8852928099999973 | 0.0
2.5600000000000026e-06 0.0002457599999999995 0.008847359999999874 0.14155775999999756 0.849346559999987 | 0.0
6.249999999999997e-06 0.0004749999999999983 0.013537499999999862 0.1714749999999923 0.8145062499999898 | 0.0
1.296e-05 0.0008121599999999975 0.019085759999999924 0.19934015999998905 0.780748959999995 | 0.0
2.401000000000002e-05 0.0012759599999999976 0.025428059999999933 0.22521995999998926 0.7480520099999917 | 0.0
4.095999999999998e-05 0.0018841599999999965 0.03250175999999992 0.2491801599999873 0.7163929599999945 | 0.0
6.560999999999994e-05 0.002653559999999992 0.040245659999999905 0.27128555999998616 0.6857496099999909 | 0.0
0.0001 0.003599999999999989 0.04859999

In [42]:
sum([10**x for x in putative_ch_priors])

27.490336077353305

In [55]:
[10**x for x in putative_ch_priors]

[9.99999998800001e-13,
 9.99999998800001e-13,
 9.99999998800001e-13,
 0.0006666666658666673,
 0.0006666666658666673,
 9.99999998800001e-13,
 9.99999998800001e-13,
 9.99999998800001e-13,
 9.99999998800001e-13,
 9.99999998800001e-13,
 9.99999998800001e-13,
 0.0013333333317333347,
 0.0006666666658666673,
 0.0006666666658666673,
 0.0006666666658666673,
 9.99999998800001e-13,
 0.0006666666658666673,
 0.0006666666658666673,
 0.0006666666658666673,
 0.0019999999976000023,
 0.0006666666658666673,
 9.99999998800001e-13,
 9.99999998800001e-13,
 0.0006666666658666673,
 0.99]

In [44]:
coverage_dict.keys()

dict_keys([0, 1, 2, 3, 4])

In [45]:
observed_proportions


[0.8166666666666667,
 0.13333333333333333,
 0.03333333333333333,
 0.016666666666666666,
 0.0]

In [5]:
from statistics import median
cells = ['PGP1_21','PGP1_22','PGP1_A1']
chambers = list(range(1,25))
total_base = 0

for cell in cells:
    print(cell)
    cell_tot = 0
    lens = []
    for ch in chambers:
        infile = 'fragment_boundary_beds/{}/ch{}.bed'.format(cell,ch)
        with open(infile,'r') as inf:
            for line in inf:
                el = line.strip().split()
                start = int(el[1])
                end   = int(el[2])
                cell_tot += end-start
                lens.append(end-start)
    
    
    print('total:  {}'.format(cell_tot))
    print('median: {}'.format(median(lens)))
    total_base += cell_tot
print("TOTAL")
print(total_base)

PGP1_21
total:  12935933000
median: 29000.0
PGP1_22
total:  12260067000
median: 24000
PGP1_A1
total:  8080167000
median: 135000.0
TOTAL
33276167000


In [4]:
4*3.2

12.8